In [4]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Load data
import load_data as data

In [5]:
# n different points we want to cluster
# We have 59 points in all 3 categories * 2 people * 10 repetitions (maia_fall only had 9)
train_trials = 59
test_trials = 16
# Tensor length of 80 since that's how many frames we decided on--easily modifiable
NUM_FRAMES = 80
NUM_FEATURES = 36

# Starting off with just one point so that it's easier to follow along with tutorials
# Will switch to more than one point later
# joint_index specifies which of the 36 joints we want, 0 is nose
# Find corresponding values at https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/output.md
joint_index = 0 

# 3D
# Xtr, Ytr = data.train_get_three_dimension(train_trials)
# Xte, Yte = data.test_get_three_dimension(test_trials)



In [6]:
# Start training
accuracies = np.zeros(NUM_FEATURES)
for i in range(NUM_FEATURES):
    # 2D
    Xtr, Ytr = data.train_get_one_keypoint(train_trials, i)
    Xte, Yte = data.test_get_one_keypoint(test_trials, i)


    # tf Graph Input

    # 3D
    # xtr = tf.placeholder("float", [train_trials, NUM_FRAMES, NUM_FEATURES])
    # xte = tf.placeholder("float", [NUM_FRAMES, NUM_FEATURES])

    # 2D
    xtr = tf.placeholder("float", [train_trials, NUM_FRAMES])
    xte = tf.placeholder("float", [NUM_FRAMES])

    # Nearest Neighbor calculation using L1 Distance
    # Calculate L1 Distance
    distance = tf.reduce_sum(tf.abs(tf.add(xtr, tf.negative(xte))), reduction_indices=1)
    # Prediction: Get min distance index (Nearest neighbor)
    pred = tf.argmin(distance, 0)

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

    ###### Training code

    with tf.Session() as sess:
        accuracy = 0.
        # Run the initializer
        sess.run(init)

        # loop over test data
        for j in range(len(Xte)):
            # Get nearest neighbor
            # 3D
            # nn_index = sess.run(pred, feed_dict={xtr: Xtr, xte: Xte[i, :, :]})

            #2D
            nn_index = sess.run(pred, feed_dict={xtr: Xtr, xte: Xte[j, :]})
            
            # Get nearest neighbor class label and compare it to its true label
#             print("Test", i, "Prediction:", np.argmax(Ytr[nn_index]) + 1, \
#                 "True Class:", np.argmax(Yte[i]) + 1)
            # Calculate accuracy
            if np.argmax(Ytr[nn_index]) == np.argmax(Yte[j]):
                accuracy += 1./len(Xte)
#         print("Done!")
#         print("Accuracy:", accuracy)
        accuracies[i] = accuracy

# plt.plot(accuracies)

In [7]:
#### Use Weighting Method to Refine Training
# Based on the information about which joints most accurately classify

# 2D
xtr = tf.placeholder("float", [train_trials, NUM_FRAMES])
xte = tf.placeholder("float", [NUM_FRAMES])

# Nearest Neighbor calculation using L1 Distance
# Calculate L1 Distance
distance = tf.reduce_sum(tf.abs(tf.add(xtr, tf.negative(xte))), reduction_indices=1)
# Prediction: Get min distance index (Nearest neighbor)
pred = tf.argmin(distance, 0)
init = tf.global_variables_initializer()

accuracies_for_tests = np.zeros(test_trials)

for i in range(test_trials):
    # for each test example, classify with one keypoint at a time -- 
    # each keypoint will vote for the label it predicts
    
    label_votes = np.zeros(3)
    
    for j in range(NUM_FEATURES):
        
        # 2D
        Xtr, Ytr = data.train_get_one_keypoint(train_trials, j)
        Xte, Yte = data.test_get_one_keypoint(test_trials, j)

        ###### Training code

        with tf.Session() as sess:
            accuracy = 0.
            # Run the initializer
            sess.run(init)

            # loop over test data

            #2D
            nn_index = sess.run(pred, feed_dict={xtr: Xtr, xte: Xte[i, :]})
            predicted = np.argmax(Ytr[nn_index])     
            label_votes[predicted - 1] = label_votes[predicted - 1] + 1
        
    winner = np.argmax(label_votes)
    if winner == np.argmax(Yte[i]):
        accuracy += 1./len(Xte)
        
    print("Test", i, "Prediction:", winner + 1, \
        "True Class:", np.argmax(Yte[i]) + 1)       
        
    accuracies_for_tests[i] = accuracy
    
print(accuracies_for_tests)



Test 0 Prediction: 1 True Class: 1
Test 1 Prediction: 2 True Class: 2
Test 2 Prediction: 1 True Class: 3
Test 3 Prediction: 1 True Class: 2
Test 4 Prediction: 2 True Class: 2
Test 5 Prediction: 2 True Class: 3
Test 6 Prediction: 3 True Class: 1
Test 7 Prediction: 3 True Class: 1
Test 8 Prediction: 1 True Class: 2
Test 9 Prediction: 1 True Class: 2
Test 10 Prediction: 1 True Class: 1
Test 11 Prediction: 2 True Class: 3
Test 12 Prediction: 1 True Class: 2
Test 13 Prediction: 2 True Class: 3
Test 14 Prediction: 1 True Class: 1
Test 15 Prediction: 1 True Class: 1
[ 0.0625  0.0625  0.      0.      0.0625  0.      0.      0.      0.      0.
  0.0625  0.      0.      0.      0.0625  0.0625]
